# 18. Implement CTR, the stream cipher mode

https://cryptopals.com/sets/3/challenges/18

https://en.wikipedia.org/wiki/Block_cipher_mode_of_operation#Counter_(CTR)

In [1]:
from cryptopals.utils import bytes_xor
from Cryptodome.Cipher import AES
import math

def generate_ctr_keystream(key: bytes, nonce: int, msglen: int) -> bytes:
    aes = AES.new(key, AES.MODE_ECB)
    keystream = b""
    for counter in range(math.ceil(msglen/AES.block_size)): # generate for N blocks covering all message 
        to_be_encrypted = nonce.to_bytes(length=AES.block_size//2, byteorder='little') + counter.to_bytes(length=AES.block_size//2, byteorder='little')
        keystream += aes.encrypt(to_be_encrypted)
    return keystream[:msglen] # trim keystream to message lenght (if shorter than N blocks 

def aes_ctr_decode_encode(b: bytes, key: bytes, nonce: int) -> bytes:
    return bytes_xor(b,generate_ctr_keystream(key,nonce,len(b)))

In [2]:
from base64 import b64decode

message = b"L77na/nrFsKvynd6HzOoG7GHTLXsTVu9qvY/2syLXzhPweyyMTJULu/6/kXX0KSvoOLSFQ=="
key=b"YELLOW SUBMARINE"
nonce=0 # "number used once"

plaintext = aes_ctr_decode_encode(b64decode(message),key,nonce)
print(f"{plaintext=}")

plaintext=b"Yo, VIP Let's kick it Ice, Ice, baby Ice, Ice, baby "


# 19. Break fixed-nonce CTR mode using substitutions

https://cryptopals.com/sets/3/challenges/19

In [3]:
strings19 = [
b"SSBoYXZlIG1ldCB0aGVtIGF0IGNsb3NlIG9mIGRheQ==",
b"Q29taW5nIHdpdGggdml2aWQgZmFjZXM=",
b"RnJvbSBjb3VudGVyIG9yIGRlc2sgYW1vbmcgZ3JleQ==",
b"RWlnaHRlZW50aC1jZW50dXJ5IGhvdXNlcy4=",
b"SSBoYXZlIHBhc3NlZCB3aXRoIGEgbm9kIG9mIHRoZSBoZWFk",
b"T3IgcG9saXRlIG1lYW5pbmdsZXNzIHdvcmRzLA==",
b"T3IgaGF2ZSBsaW5nZXJlZCBhd2hpbGUgYW5kIHNhaWQ=",
b"UG9saXRlIG1lYW5pbmdsZXNzIHdvcmRzLA==",
b"QW5kIHRob3VnaHQgYmVmb3JlIEkgaGFkIGRvbmU=",
b"T2YgYSBtb2NraW5nIHRhbGUgb3IgYSBnaWJl",
b"VG8gcGxlYXNlIGEgY29tcGFuaW9u",
b"QXJvdW5kIHRoZSBmaXJlIGF0IHRoZSBjbHViLA==",
b"QmVpbmcgY2VydGFpbiB0aGF0IHRoZXkgYW5kIEk=",
b"QnV0IGxpdmVkIHdoZXJlIG1vdGxleSBpcyB3b3JuOg==",
b"QWxsIGNoYW5nZWQsIGNoYW5nZWQgdXR0ZXJseTo=",
b"QSB0ZXJyaWJsZSBiZWF1dHkgaXMgYm9ybi4=",
b"VGhhdCB3b21hbidzIGRheXMgd2VyZSBzcGVudA==",
b"SW4gaWdub3JhbnQgZ29vZCB3aWxsLA==",
b"SGVyIG5pZ2h0cyBpbiBhcmd1bWVudA==",
b"VW50aWwgaGVyIHZvaWNlIGdyZXcgc2hyaWxsLg==",
b"V2hhdCB2b2ljZSBtb3JlIHN3ZWV0IHRoYW4gaGVycw==",
b"V2hlbiB5b3VuZyBhbmQgYmVhdXRpZnVsLA==",
b"U2hlIHJvZGUgdG8gaGFycmllcnM/",
b"VGhpcyBtYW4gaGFkIGtlcHQgYSBzY2hvb2w=",
b"QW5kIHJvZGUgb3VyIHdpbmdlZCBob3JzZS4=",
b"VGhpcyBvdGhlciBoaXMgaGVscGVyIGFuZCBmcmllbmQ=",
b"V2FzIGNvbWluZyBpbnRvIGhpcyBmb3JjZTs=",
b"SGUgbWlnaHQgaGF2ZSB3b24gZmFtZSBpbiB0aGUgZW5kLA==",
b"U28gc2Vuc2l0aXZlIGhpcyBuYXR1cmUgc2VlbWVkLA==",
b"U28gZGFyaW5nIGFuZCBzd2VldCBoaXMgdGhvdWdodC4=",
b"VGhpcyBvdGhlciBtYW4gSSBoYWQgZHJlYW1lZA==",
b"QSBkcnVua2VuLCB2YWluLWdsb3Jpb3VzIGxvdXQu",
b"SGUgaGFkIGRvbmUgbW9zdCBiaXR0ZXIgd3Jvbmc=",
b"VG8gc29tZSB3aG8gYXJlIG5lYXIgbXkgaGVhcnQs",
b"WWV0IEkgbnVtYmVyIGhpbSBpbiB0aGUgc29uZzs=",
b"SGUsIHRvbywgaGFzIHJlc2lnbmVkIGhpcyBwYXJ0",
b"SW4gdGhlIGNhc3VhbCBjb21lZHk7",
b"SGUsIHRvbywgaGFzIGJlZW4gY2hhbmdlZCBpbiBoaXMgdHVybiw=",
b"VHJhbnNmb3JtZWQgdXR0ZXJseTo=",
b"QSB0ZXJyaWJsZSBiZWF1dHkgaXMgYm9ybi4="]

In [4]:
import os
key19 = os.urandom(AES.block_size)
nonce = 0
ciphers19 = [ aes_ctr_decode_encode(b64decode(s),key19,nonce) for s in strings19 ]

## Attack

> Because the CTR nonce wasn't randomized for each encryption, each ciphertext has been encrypted against the same keystream. (...) the actual "encryption" of a byte of data boils down to a single XOR operation

I can try to reconstruct the keystream by using frequency analysis on all bytes in the same position from all 40 ciphers, using the same approach from challenge 3. One issue will be that this will work fine for the first bytes (for which I have a lot of ciphers) but fails for the last bytes of longer ciphers...

In [65]:
freqs_initials = {
"A":	11.7,	
"B":	4.4,	
"C":	5.2	, 
"D":	3.2	,
"E":	2.8	,
"F":	4	,
"G":	1.6	,
"H":	4.2	,
"I":	7.3	,
"J":	0.51,	
"K":	0.86,	
"L":	2.4	,
"M":	3.8	,
"N":	2.3	,
"O":	7.6	,
"P":	4.3	,
"Q":	0.22,	
"R":	2.8	,
"S":	6.7	,
"T":	16	,
"U":	1.2	,
"V":	0.82,	
"W":	5.5	,
"X":	0.045,	
"Y":	0.76,	
"Z":	0.045,
}

freqs_initials = { l : f*0.01 for l,f in freqs_initials.items()} 

In [66]:
from cryptopals.utils import freqs_letters, score_text, crack_single_xor

def find_ctr_keystream(ciphers):
    maxlen = max( [len(c) for c in ciphers])
    
    cipcols = [ b"" ] * maxlen
    for c in ciphers:
        for i,b in enumerate(c):
            cipcols[i] += b.to_bytes()
    
    keystream = b""
    for i,cc in enumerate(cipcols):
        if i==0:
            best_guess = crack_single_xor(cc,freqs_initials)
        else:
            best_guess = crack_single_xor(cc,freqs_letters)
        keystream += best_guess[2]

    return keystream

In [67]:
keystream19 = find_ctr_keystream(ciphers19)
for c in ciphers19:
    print(bytes_xor(c,keystream19))

b'i have met them at close of&,a&'
b'coming with vivid faces'
b'from counter or desk among a:e&'
b'eighteenth-century houses.'
b'i have passed with a nod of&<h:\x80\x08\t\x00\x00'
b'or polite meaningless words*'
b'or have lingered awhile and&;a6\xc4'
b'polite meaningless words,'
b'and thought before I had doh-'
b'of a mocking tale or a gibe'
b'to please a companion'
b'around the fire at the club*'
b'being certain that they and&\x01'
b'but lived where motley is wi:ne'
b'all changed, changed utterl\x7fr'
b'a terrible beauty is born.'
b"that woman's days were spenr"
b'in ignorant good will,'
b'her nights in argument'
b'until her voice grew shrill('
b'what voice more sweet than n-r,'
b'when young and beautiful,'
b'she rode to harriers?'
b'this man had kept a school'
b'and rode our winged horse.'
b'this other his helper and ft!e1\xc4'
b'was coming into his force;'
b'he might have won fame in tn- :\xce\x04@'
b'so sensitive his nature seek-ds'
b'so daring and sweet his thos/h+\x8e'
b'this oth

### Notes

* Why is the first letter lowecase (and it's uppercase in the original strings)?

# 20. Break fixed-nonce CTR statistically

https://cryptopals.com/sets/3/challenges/20

> Take your collection of ciphertexts and truncate them to a common length (the length of the smallest ciphertext will work).

I could do this, or I could use the approach used for challenge 19 and try to exploit the smaller stastistics also for the longer messages: in many cases I can do better!

### Notes

* The first character is never recovered: why?!
    * The first character is always a letter (no space or puctuation!) so the frequency is likely different that in normal text. I tried to implement this, but it does not solve the issue (?)
    * The fact that the first letter in Challenge 19 (solved in the same way) is always lowercase adds to the weirdness of the result... 
* Even a smaller statistics can lead to a correct single XOR crack, so no real reason not to attempt and truncate the data to the smallest ciphertext!

In [68]:
key20 = os.urandom(AES.block_size)

with open("input/20.txt") as f:
    strings20 = [ b64decode(l.strip()) for l in f.readlines() ]

ciphers20 = [ aes_ctr_decode_encode(p,key20,nonce) for p in strings20 ]

In [69]:
minlen = min([len(c) for c in ciphers20])
ciphers20_trunc = [c[:minlen] for c in ciphers20]

#keystream20 = find_ctr_keystream(ciphers20_trunc)
keystream20 = find_ctr_keystream(ciphers20)

In [70]:
plaintext20 = [ bytes_xor(c,keystream20) for c in ciphers20] 

for p,s in zip(plaintext20,strings20):
    print(f"Original  : {s.decode()}")
    print(f"Recovered : {p.decode()}")
    print()

Original  : I'm rated "R"...this is a warning, ya better void / Poets are paranoid, DJ's D-stroyed
Recovered : n'm rated "R"...this is a wasning, ya better void / Poets are paranoid,!DJ's&D-stroyed

Original  : Cuz I came back to attack others in spite- / Strike like lightnin', It's quite frightenin'!
Recovered : duz I came back to attack otiers in spite- / Strike like lightnin', It'r quire frightenin' 

Original  : But don't be afraid in the dark, in a park / Not a scream or a cry, or a bark, more like a spark;
Recovered : eut don't be afraid in the d`rk, in a park / Not a scream or a cry, or ` barm, more like a!pa$n

Original  : Ya tremble like a alcoholic, muscles tighten up / What's that, lighten up! You see a sight but
Recovered : ~a tremble like a alcoholic,!muscles tighten up / What's that, lighten tp! Yiu see a sight!nut

Original  : Suddenly you feel like your in a horror flick / You grab your heart then wish for tomorrow quick!
Recovered : tuddenly you feel like your hn a h